In [ ]:
import databricks.sdk
print(databricks.sdk.version.__version__)
# 最新版(0.18)でなければ下でpipを実行する

0.18.0


In [ ]:
%pip install --upgrade databricks-sdk

In [ ]:
dbutils.library.restartPython()

In [ ]:
import os, time, pprint, csv
from databricks.sdk.service import iam

In [ ]:
# ワークスペース用クライアントを作成、クラスター再起動のたびに実行してください

from databricks.sdk import WorkspaceClient

w = WorkspaceClient(host="https://adb-6577950137824096.16.azuredatabricks.net", token="")


In [ ]:
from databricks.sdk import AccountClient

a = AccountClient(
  host                = "https://accounts.azuredatabricks.net",
  account_id          = "26b95558-3fb8-477d-a1f8-8603c2c64d5c",
  azure_tenant_id     = "40e7b67c-11c2-470f-bb62-0ff7a5accad5",
  azure_client_id     = "342aa662-89ee-4616-ba9d-360ef93a87d9",
  azure_client_secret = "",
  auth_type           = "azure-client-secret"
)


In [ ]:
# ここのワークスペースのID
WORKSPACE_ID = 6577950137824096

In [ ]:
def create_user(user_name,display_name):
    u = a.users.create(user_name=user_name,display_name=display_name)
    a.workspace_assignment.update(permissions = [iam.WorkspacePermission.USER], workspace_id=WORKSPACE_ID, principal_id=u.id)
    return u

In [ ]:
# ユーザーリストの読み込み
user_file = 'userlist.csv'

users = []
with open(user_file, mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row['氏名'],row['アカウント名'])
        users.append(row)

In [ ]:
# リストを基にしてのユーザー作成
for user in users:
    u = create_user(user_name=user['アカウント名'],display_name=user['氏名'])

In [ ]:
# カタログスキーマの自動作成と権限付与

# 処理するIDの始点と終点を指定
startID = 1032
endID = 1040

for i in range(startID, endID+1):
    usrname = f'adbuser{i:04}'
    w.schemas.create(name=usrname,catalog_name='ws_fduacmp_001')
    spark.sql(f'GRANT ALL PRIVILEGES ON SCHEMA ws_fduacmp_001.{usrname} TO `{usrname}@fdua2024kc.onmicrosoft.com`')